# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,prado,-17.3411,-39.2208,25.57,82,56,3.39,BR,1712616544
1,1,ribeira grande,38.5167,-28.7000,17.10,96,100,6.71,PT,1712616463
2,2,port-aux-francais,-49.3500,70.2167,8.18,79,86,15.16,TF,1712616453
3,3,ta`u,-14.2336,-169.5144,29.36,78,20,6.19,AS,1712616544
4,4,sao joao da barra,-21.6403,-41.0511,25.05,83,88,4.37,BR,1712616544


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 1,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

c:\Users\lucpe\anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
niceweather_cities = city_data_df[(city_data_df['Cloudiness']==0) & (city_data_df['Max Temp'] <27) & (city_data_df['Max Temp'] >21) & (city_data_df['Wind Speed'] <4)]

niceweather_cities


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,vila velha,-20.3297,-40.2925,26.08,84,0,2.57,BR,1712616372
87,87,tamanrasset,22.7850,5.5228,24.95,9,0,3.09,DZ,1712616552
275,275,shirgaon,19.7000,72.7000,25.75,74,0,1.30,IN,1712616571
322,322,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1712616576
495,495,seeb,23.6703,58.1891,26.05,44,0,2.57,OM,1712616593


In [5]:
# Drop any rows with null values
drop_null = niceweather_cities.isnull().sum()

# Display sample data
print(drop_null)

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = niceweather_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
78,vila velha,BR,-20.3297,-40.2925,84,
87,tamanrasset,DZ,22.7850,5.5228,9,
275,shirgaon,IN,19.7000,72.7000,74,
322,saint-pierre,RE,-21.3393,55.4781,73,
495,seeb,OM,23.6703,58.1891,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit": 20
}
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
 
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
  
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
vila velha - nearest hotel: Hotel Prainha
tamanrasset - nearest hotel: فندق الأمان
shirgaon - nearest hotel: KD Residency
saint-pierre - nearest hotel: Tropic Hotel
seeb - nearest hotel: فندق جولدن كراون السيب


,City,Country,Lat,Lng,Humidity,Hotel Name
78,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
87,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
275,shirgaon,IN,19.7000,72.7000,74,KD Residency
322,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
495,seeb,OM,23.6703,58.1891,44,فندق جولدن كراون السيب


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Filter hotel_df to drop rows where "No hotel found" are in "Hotel Name"
best_hotels_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Configure the map plot
map_plot_2 = best_hotels_df.hvplot.points(
    x='Lng', 
    y= 'Lat',
    geo=True,
    color='City',
    size=100,
    alpha=0.5,
    tiles= 'OSM',
    hover_cols= ['City', 'Hotel Name'])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)